In [ ]:
import pandas as pd
import sqlite3
from datetime import datetime

loc = "./data/"

call_logs = sqlite3.connect(loc + "call_log.db")
call_logs_cursor = call_logs.cursor()

call_log_documents = pd.DataFrame(columns=[["Number", "Owner", "Time", "Duration", "Type", "CountryISO",]])
for i, c in enumerate(call_logs_cursor.execute("SELECT * FROM CALLS").fetchall()):
    call_log_documents.loc[i] = [ c[1], c[12], datetime.fromtimestamp(int(c[5])//1000).strftime("%d-%m-%Y %H:%M:%S"), c[6], "INCOMING" if c[8]==1 else "OUTGOING" if c[8] == 2 else "MISSED" if c[8] == 3 else "REJECTED", c[19] ]
    
call_log_documents

sms = sqlite3.connect(loc + "sms.db")
sms_cursor = sms.cursor()

sms_documents = pd.DataFrame(columns = ["Address", "Body", "Date Sent", "Date Received", "Type", "Seen"])
for i, c in enumerate(sms_cursor.execute("SELECT * FROM SMS").fetchall()):
    sms_documents.loc[i] = [ c[2], c[12], datetime.fromtimestamp(int(c[5])/1000).strftime("%d-%m-%Y, %H:%M:%S") if c[9] == 1 else datetime.fromtimestamp(int(c[4])/1000).strftime("%d-%m-%Y, %H:%M:%S"), datetime.fromtimestamp(int(c[4])/1000).strftime("%d-%m-%Y, %H:%M:%S"), "Received" if c[9] == 1 else "Sent", "True" if c[18] == 1 else "False" ]

sms_documents

In [2]:
from transformers import pipeline
import torch, sqlite3
import asyncio

device = "cuda:0" if torch.cuda.is_available() else "cpu"
batch_size = 12
language = "en"
gemma = "google/gemma-2-2b-it"
nlp = gemma
nlp_model = None
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

def load_NLP():
    global nlp_model
    nlp_model = pipeline("text-generation", model=nlp, model_kwargs={"torch_dtype": torch_dtype}, device=device)
    if nlp_model != None:
        print(f"{nlp} Loaded!")

load_NLP()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cpu


google/gemma-2-2b-it Loaded!


In [3]:
def generate_query(query):
    try:
        messages = [
            {
                "role": "user", "content": 
                f"""
                Context: You are a SQL generator. 
                Given the following database schema:
                    Table: sms alias Messages
                    Columns:
                    - id (integer)
                    - Address (text)
                    - Date Sent (date)
                    - Date Received (date)
                    - Type (text)
                    - Body (text)
                    - Seen (boolean)
                    
                    Table: Contacts
                    Columns:
                    - id (integer)
                    - name (text)
                    - number (number)
                    - email (text)
                    
                    Table: Call Logs
                    Columns:
                    - id (integer)
                    - Owner (text)
                    - Date Time (number)
                    - Duration (number)
                    - Type (text)
                    
                    Table: Files
                    Columns:
                    - path (text)
                    - name (text)
                    - parent (text)
                    - size (number)
                    - datetime (datetime)
                    - ext (text) alias type
                Convert the following user question into a correct, safe SQL query. 
                Return only SQL, no explanations.
                Query: {query}
                """
            }
        ]

        outputs = nlp_model(messages, max_new_tokens=128)
        assistant_response = outputs[0]["generated_text"][-1]["content"].strip()
        return assistant_response
    
    except Exception as e:
        print(e)
        
    return 

def run_query(query):
    query = query[6:-3]

    if str(query).lower().startswith("select"):
        return
    conn = sqlite3.connect("./data/sms.db")
    cur = conn.cursor()
    output = cur.execute(
        query,
        []
    ).fetchall()
    return output
    
def convert_to_nlp(results):
    messages = [
        {
            "role": "user", "content": 
            f"""
            Data: {results}
            Convert the following data in human readable format
            """
        }
    ]
    try:
        outputs = nlp_model(messages, max_new_tokens=256)
        assistant_response = outputs[0]["generated_text"][-1]["content"].strip()
        return assistant_response
        
    except Exception as e:
        print(e)
        
def hello():
    query = "get all messages"
    print(query)

    sql_query = generate_query(query)
    print("SQL:", sql_query)

    results = run_query(sql_query)
    print("Results: ", results)

    output = convert_to_nlp(results)
    print("Output: ", output)

hello()

get all messages
SQL: ```sql
SELECT * FROM sms;
```
Results:  [(1, 1, '+16505551212', 2, 1763649906518, 1763649911000, 0, 1, -1, 1, 0, 'proto:CjoKImNvbS5nb29nbGUuYW5kcm9pZC5hcHBzLm1lc3NhZ2luZy4SFCIAKhCZ3eMPw/NJNq4kblZMS0n5', 'Morning Bob! You awake?', None, 0, 1, -1, 'com.google.android.apps.messaging', 1), (2, 1, '+16505551212', None, 1763649943149, 0, None, 1, -1, 2, None, 'proto:CjoKImNvbS5nb29nbGUuYW5kcm9pZC5hcHBzLm1lc3NhZ2luZy4SFCIAKhDjjU8W09NCgKk5bzpsDr6K', 'Barely. Need coffee first.', None, 0, 1, -1, 'com.google.android.apps.messaging', 1), (3, 1, '+16505551212', 2, 1763649952769, 1763649958000, 0, 1, -1, 1, 0, 'proto:CjoKImNvbS5nb29nbGUuYW5kcm9pZC5hcHBzLm1lc3NhZ2luZy4SFCIAKhC39iDFbgNOfK+qd3qgZC1F', 'Same. Rough night?', None, 0, 1, -1, 'com.google.android.apps.messaging', 1), (4, 1, '+16505551212', None, 1763649974076, 0, None, 1, -1, 2, None, 'proto:CjoKImNvbS5nb29nbGUuYW5kcm9pZC5hcHBzLm1lc3NhZ2luZy4SFCIAKhC64ELCLLRCTadlK7lUWGf8', 'I stayed up fixing that server issue.', None

In [23]:
from transformers import pipeline
import whisperx

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

class ASR:
    def __init__(self, device, compute_type):
        self.model = None
        self.__asr = "large-v3"
        self.load_model(device, compute_type)
    
    def load_model(self, device, compute_type):
        self.model = whisperx.load_model(self.__asr, device, compute_type=str(torch_dtype).split(".")[1])
        if self.model != None:
            print(f"{self.__asr} Loaded!")
            
    def isLoaded(self):
        return False if self.model == None else True
    
class NLP:
    def __init__(self, device, compute_type):
        self.__nlp = "google/gemma-2-2b-it"
        self.model = None
        self.load_model(device, compute_type)
    
    def load_model(self, device, compute_type):
        self.model = pipeline("text-generation", model=self.__nlp, model_kwargs={"torch_dtype": torch_dtype}, device=device)
        if self.model != None:
            print(f"{self.__nlp} Loaded!")
        
    def isLoaded(self):
        return False if self.model == None else True
        
class ZSC:
    def __init__(self, device, compute_type):
        self.__zsc = "MoritzLaurer/deberta-v3-large-zeroshot-v2.0"
        self.model = None
        self.load_model(device, compute_type)
    
    def load_model(self, device, compute_type):
        self.model = pipeline("zero-shot-classification", model=self.__zsc)
        if self.model != None:
            print(f"{self.__zsc} Loaded!")
        
    def isLoaded(self):
        return False if self.model == None else True
        
#asr = ASR().load_model(device, torch_dtype)
#NLP.load_model()
asr = ASR(device, torch_dtype)
nlp = NLP(device, torch_dtype)
zsc = ZSC(device, torch_dtype)

e:\REPOSITORIES\WEB\Forensix\.venv\Lib\site-packages\ctranslate2\__init__.py:8: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
e:\REPOSITORIES\WEB\Forensix\.venv\Lib\site-packages\pyannote\audio\core\io.py:212: UserWarning: torchaudio._backend.list_audio_backends has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  torchaudio.list_audio_backends()
e:\REPOSITORIES\WEB\Forensix\.venv\Lib\site-packages\speechbrain\utils\torch_audio_backend.py:57: UserWarning: torchaudio

2025-11-24 23:19:21 - whisperx.asr - INFO - No language specified, language will be detected for each audio file (increases inference time)
2025-11-24 23:19:21 - whisperx.vads.pyannote - INFO - Performing voice activity detection using Pyannote...


E:\Program Files\Python\Python313\Lib\inspect.py:1020: UserWarning: Module 'speechbrain.pretrained' was deprecated, redirecting to 'speechbrain.inference'. Please update your script. This is a change from SpeechBrain 1.0. See: https://github.com/speechbrain/speechbrain/releases/tag/v1.0.0
  if ismodule(module) and hasattr(module, '__file__'):
Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.5.6. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint e:\REPOSITORIES\WEB\Forensix\.venv\Lib\site-packages\whisperx\assets\pytorch_model.bin`
e:\REPOSITORIES\WEB\Forensix\.venv\Lib\site-packages\pyannote\audio\core\io.py:212: UserWarning: torchaudio._backend.list_audio_backends has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec.

Model was trained with pyannote.audio 0.0.1, yours is 3.4.0. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.8.0+cpu. Bad things might happen unless you revert torch to 1.x.
large-v3 Loaded!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cpu


google/gemma-2-2b-it Loaded!


Device set to use cpu


MoritzLaurer/deberta-v3-large-zeroshot-v2.0 Loaded!


In [3]:
import sqlite3

sqlite = sqlite3.connect("./test.db")
cur = sqlite.cursor()
cur.execute("DROP TABLE IF EXISTS MESSAGES;")
cur.execute("DROP TABLE IF EXISTS CALL_LOGS;")
cur.execute("DROP TABLE IF EXISTS CONTACTS;")
cur.execute("DROP TABLE IF EXISTS FILES;")
cur.execute(
    '''
    CREATE TABLE IF NOT EXISTS MESSAGES(
        ADDRESS VARCHAR(100) NOT NULL,
        BODY VARCHAR(512) NOT NULL,
        DATE_SENT DATETIME NOT NULL,
        DATE_RECEIVED DATETIME NOT NULL,
        TYPE VARCHAR(10) NOT NULL,
        SEEN VARCHAR(4) NOT NULL
    );
    '''
)
cur.execute(
    '''
    CREATE TABLE IF NOT EXISTS CALL_LOGS(
        NUMBER VARCHAR(10) NOT NULL,
        DATE DATETIME NOT NULL,
        DURATION INT NOT NULL,
        TYPE VARCHAR(10) NOT NULL
    );
    '''
) 
cur.execute(
    '''
    CREATE TABLE IF NOT EXISTS CONTACTS(
        NAME VARCHAR(50) NOT NULL,
        NUMBER VARCHAR(10) NOT NULL,
        GROUP_ID INT NOT NULL,
        EMAIL VARCHAR(255) NOT NULL
    );
    '''
)
cur.execute(
    '''
    CREATE TABLE IF NOT EXISTS FILES(
        PATH VARCHAR(260) NOT NULL PRIMARY KEY,
        NAME VARCHAR(255) NOT NULL,
        DIRECTORY VARCHAR(255) NOT NULL,
        SIZE INT NOT NULL,
        DATETIME DATETIME NOT NULL,
        EXT VARCHAR(5) NOT NULL
    );
    '''    
) 
sms = [
    ["7899977860","Oh I’m shaking. You talk big but all I see is","21-11-2025 01:01:17","21-11-2025 01:01:17","Received","True"],
    ["9999999999","We don’t bow to tyrants hiding behind darkness","21-11-2025 00:59:43","21-11-2025 00:59:43","Sent","True"]
]
cur.executemany(
    '''
    INSERT INTO MESSAGES(ADDRESS, BODY, DATE_SENT, DATE_RECEIVED, TYPE, SEEN) VALUES(?, ?, ?, ?, ?, ?);
    '''
    , sms
)
sqlite.commit()
cur.close()
sqlite.close()

In [3]:
import sqlite3

sqlite = sqlite3.connect("../../Web/Forensix.db")
cur = sqlite.cursor()
cur.execute("DELETE FROM EVIDENCES")
cur.close()
sqlite.commit()
sqlite.close()

In [4]:
import torch

torch.cuda.is_available()

False